<div><center>
<table><tr>
<td>
<img src="images/Embraer_logo.png" alt="Empresa Brasileira de Aeronáutica (Embraer)" style="width: 300px;"/>
    </td>
<td>
<img src="images/ITA_logo.png" alt="Instituto Tecnológico da Aeronáutica (ITA)" style="width: 300px;"/>
    </td>
    </tr></table
</center></div>

In [ ]:
import model_reference as mf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

plt.style.use('seaborn')
warnings.filterwarnings('ignore')


# EMBRAER | ITA

## ENGENHARIA AERONÁUTICA E MECÂNICA - PEE 30

### AC-701 : SISTEMAS PROPULSIVOS - PROF. LACAVA

### Autores:

* Camilla Santos
* Eloi Antonio
* Gabriel Henrique
* Jonas Degrave
* Lucas Schroeder
* Matheus Monteverde
* Orlando Gabriel
* Rafael Bayão

---

## Exercício 1: Montagem do Simulador

* Matheus Monteverde
* Rafael Bayão

### Estratégia para criação do simulador

O simulador foi montado a partir de uma arquitetura de Programação Orientada a Objetos. Dessa forma, foram modelados processos que são herdados pelos componentes e esses são utilizados pelos motores a medida da necessidade. Os a lista de objetos e suas funções podem ser visualizados abaixo:

Processos térmicos: cada processo armazena as propriedades adequadas para formulação de suas transformações.

* ThermalProcess: Classe principal - implementa métodos genéricos isentrópicos e é herdada pelas demais.
* StaticThermalProcess: implementa métodos para processos com entrada e saída com velocidades desprezíveis para o gás.
* SpeedInThermalProcess: implementa métodos para processos com entrada a velocidade considerável e saída a velocidade desprezível.
* SpeedOutThermalProcess: implementa métodos para processos com entrada a velocidade desprezível e saída a velocidade consideravel.

<center><img src="images/questao1/thermal-processes.png" alt="Thermal Process"/></center>

Componentes: cada componente herda um processo térmico adequado ao seu funcionamento implementa as equações necessárias. Os valores de entrada são armazenados e os de saída calculados ao serem invocados. Por fim, implementam métodos para variar a rotação e resumir todos os parametros inclusos.

* Diffuser_Adiab
* Nozzle_Adiab
* Compressor
* Fan
* Turbine
* FanTurbine
* FreeTurbine
* CombustionChamber

Motores: cada motor instancia sequencialmente os componentes necessários para o funcionamento e implementa métodos para ajustar globalmente a rotação e fornecer todas as propriedades de entrada e saída dos componentes e do próprio motor.

* TurboJet
* TurboFan
* TurboProp

<div><center>
<table><tr>
<td>
<img src="images/questao1/turbojet-schema.png" alt="turbojet-schema" style="height: 400px;"/>
    </td>
<td>
<img src="images/questao1/turbofan-schema.png" alt="turbofan-schema" style="height: 400px;"/>
    </td>
    <td>
<img src="images/questao1/turboprop-schema.png" alt="turboprop-schema" style="height: 400px;"/>
    </td>
    </tr></table
</center></div>

---

### Exemplo 2.1: TurboJet

#### Informações do exercício:

Calcular o empuxo específico e consumo específico (TSFC) para um motor turbojato operando a Mach 0,85 e a 12.200 m (Pa = 18,75kPa e Ta = 216,7 K). Usar os dados abaixo para eficiências e razões de calor específico.

| Componente | Eficiência Adiabática | $\gamma$ |
| :--------- |:---------------------:| -------- |
| difusor    | 0,97                  | 1,40     |
| compressor | 0,85                  | 1,37     |
| combustão  | 1,00                  | 1,35     |
| turbina    | 0.90                  | 1,33     |
| bocal      | 0,98                  | 1,36     |

Dados operacionais


| Dados operacionais:                         | Valor             |
| ------------------------------------------- | ----------------- |
| Razão de Pressão no compressor              | 30                |
| Temperatura na sáida da câmara de combustão | 1600 K            |
| Poder Calorífico Inferior do Combustível    | 45000 kJ/kg       |
| R Médio                                     | 288,3 m^2/(s^2.K) |
| Cp no combustor                             | 1,11 kJ/(kg.K)    |



#### Comparação de resultados

| Parâmetro                                      | Unidade   | Modelo   | Referência do material | $\Delta$ em % |
| ---------------------------------------------- | --------- | -------- | ---------------------- | ------------- |
| Temp. total no final da entrada de ar          | K         | 248.013  | 248,013                | 0             |
| Press. total no final da entrada de ar         | kPa       | 296.7483 | 29,6748                | 0             |
| Press. Total na saída do Compressor            | kPa       | 890,245  | 890,245                | 0             |
| Temp. total na saída do Compressor             | K         | 687,349  | 687,349                | 0             |
| Razão Combustível/Ar                           | -         | 0,02344  | 0,02344                | 0             |
| Temp. total na saída da turbina do compressor  | K         | 1160,66  | 1160,66                | 0             |
| Press. total na saída da turbina do compressor | kPa       | 205,316  | 205,316                | 0             |
| Velocidade de saída                            | m/s       | 1078,3   | 1078,3                 | 0             |
| Consumo Específico                             | kg/(kN.s) | 0,0275   | 0,0275                 | 0             |
| Velocidade de vôo                              | m/s       | 251,382  | 251,382                | 0             |
| Empuxo Específico                              | kN.s/kg   | 0,85219  | 0,85219                | 0             |


---

### Exemplo 2.2: TurboFan

#### Informações do exercício

Calcular o empuxo específico e consumo específico (TSFC) para um motor turbofan operando a Mach 0,85 e a 12.200 m (Pa = 18,75 e Ta = 216,7 K). Usar os dados abaixo para eficiências e razões de calor específico. Comparar com exemplo anterior para o motor turbojato.

| Componente             | Eficiências Adiabática | $\gamma$ |
| ---------------------- |:-----------:| -------- |
| Entrada de ar          | 0,97        | 1,40     |
| Fan                    | 0,85        | 1,40     |
| Compressor             | 0,85        | 1,37     |
| Câmara de combustão    | 1,00        | 1,35     |
| Turbina do compressor  | 0.90        | 1,33     |
| Turbina do fan         | 0.90        | 1,33     |
| Bocal de gases quentes | 0,98        | 1,36     |
| Bocal do fan           | 0,98        | 1,40     |

Dados operacionais:

| Parâmetro                                   | Valores         |
| ------------------------------------------- | --------------- |
| Temperatura na sáida da câmara de combustão | 1600 K          |
| Razão de Pressão no compressor              | 20              |
| Razão de Pressão no Fan                     | 1.5             |
| Razão de Passagem                           | 5.0             |
| Poder Calorífico Inferior do Combustível    | 45000 kJ/kg     |
| R Médio                                     | 288,3 m2/(s2.K) |
| Cp no combustor                             | 1,11 kJ/(kg.K)  |



#### Comparação de resultados do modelo do Turbo Fan com o material

| Parâmetro                                      | Unidade   | Modelo  | Referência do material | $\Delta$ em % |
| ---------------------------------------------- | --------- | ------- | ---------------------- | ------------- |
| Temp. total no final da entrada de ar          | K         | 248,013 | 248,013                | 0             |
| Press. total no final da entrada de ar         | kPa       | 29,6748 | 29,6748                | 0             |
| Press. Total na saída do Fan                   | kPa       | 29,6748 | 29,6748                | 0             |
| Temp. total na saída do Fan                    | K         | 283,851 | 283,851                | 0             |
| Press. Total na saída do Compressor            | kPa       | 890,245 | 890,245                | 0             |
| Temp. total na saída do Compressor             | K         | 699,879 | 699,879                | 0             |
| Razão Combustível/Ar                           | -         | 0,02312 | 0,02312                | 0             |
| Temp. total na saída da turbina do Compressor  | K         | 1183,97 | 1183,97                | 0             |
| Press. total na saída da turbina do Compressor | kPa       | 225,281 | 225,281                | 0             |
| Temp. total na saída da turbina do Fan         | K         | 968,945 | 968,945                | 0             |
| Press. total na saída da turbina do Fan        | kPa       | 90,8277 | 90,8277                | 0             |
| Velocidade de saída - Bocal do fan             | m/s       | 350,532 | 350,532                | 0             |
| Velocidade de saída - Bocal de gases quentes   | m/s       | 840,335 | 840,335                | 0             |
| Velocidade de vôo                              | m/s       | 251,382 | 251,382                | 0             |
| Empuxo Específico                              | kN.s/kg   | 1,10413 | 1,10413                | 0             |
| Consumo Específico                             | kg/(kN.s) | 0,02094 | 0,02094                | 0             |



---

### Exemplo 2.3: TurboFan - variação da rotação a partir do ponto de projeto

#### Informações do exercício

Condição de Projeto do Motor - Máxima Rotação - Nível do mar - 288,15K

| Parâmetros                                  | Valores  |
| ------------------------------------------- | -------- |
| Vazão de ar                                 | 756 kg/s |
| Tenoeratura na saída da câmara de combustão | 1550 K   |
| Razão de Pressão no compressor              | 17.2     |
| Razão de Pressão no Booster                 | 1.00     |
| Razão de Pressão no Fan                     | 1.69     |
| Razão de Passagem                           | 4.69     |
| % de perda de pressão taotal na câmara      | 5 %      |

Condições de Operação:

| Condições de operação em voo      | Valores    |
| --------------------------------- | ---------- |
| Mach de voo                       | 0          |
| Pressão atmosférica               | 101.63 kPa |
| Temperatura atmosférica           | 290 K      |
| % da rotação máxima no compressor | 1.00       |
| % da rotação máxima no fan        | 1.00       |



Eficiências e valores de $\gamma$:

| Componente             | Eficiências Adiabática | $\gamma$ |
| ---------------------- |:----------------------:| -------- |
| Entrada de ar          | 0,97                   | 1,40     |
| Fan                    | 0,93                   | 1,40     |
| Compressor             | 0,90                   | 1,37     |
| Câmara de combustão    | 0,9995                 | 1,35     |
| Turbina do compressor  | 0.95                   | 1,33     |
| Turbina do fan         | 0.932                  | 1,33     |
| Bocal de gases quentes | 0,98                   | 1,36     |
| Bocal do fan           | 0,98                   | 1,40     |


Dados termodinâmicos:

| Dados termodinâmicos                     | Valores         |
| ---------------------------------------- | --------------- |
| Poder Calorífico Inferior do Combustível | 45000 kJ/kg     |
| R Médio                                  | 288,3 m2/(s2.K) |
| Cp no combustor                          | 1,11 kJ/(kg.K)  |



#### Solução

| Parâmetro                  |        1.0 |        0.9 |       0.8 |        0.7 |       0.6 |       0.5 |
| :------------------------- | ---------: | ---------: | --------: | ---------: | --------: | --------: |
| Rotação do Fan N1          |          1 |    0.87016 |    0.7285 |    0.58684 |   0.44518 |   0.30352 |
| Empuxo Total               |    253.464 |    168.517 |   90.0758 |    48.2752 |   22.8478 |    12.024 |
| TSFC                       | 0.00996902 | 0.00911273 | 0.0092772 | 0.00898977 | 0.0100623 | 0.0131675 |
| Prf                        |       1.69 |    1.56341 |   1.38538 |    1.24073 |   1.12948 |   1.05162 |
| Prc                        |       17.2 |    14.3017 |   10.9042 |    7.59124 |   4.98958 |   3.72595 |
| Taxa de comp. Global       |     29.068 |    22.3593 |   15.1064 |    9.41871 |   5.63565 |   3.91828 |
| Temp. Saída do Combustor   |    1529.52 |    1345.31 |   1166.82 |    1013.36 |   884.936 |   781.537 |
| Vazão mássica de ar quente |    131.056 |    95.2792 |   63.0639 |    38.8194 |   23.4889 |   18.8928 |
| Consumo                    |    2.52679 |    1.53565 |  0.835651 |   0.433983 |  0.229901 |  0.158326 |
| Razão de Passagem          |      4.749 |    5.38464 |   6.01871 |    6.49411 |   6.81085 |   6.96894 |

<center><img src="images/questao1/ex2_3.jpg" alt="turboprop-schema" style="width:1000px"/></center>

#### Comparação de resultados

Para facilitar a comparação de resultados optou-se por gerar respostas para os mesmos pontos que rotação que os apresentados no material de referência. Além disso, foi utilizado os mesmos valores nos eixos e grade. Por não ter os valores exatos no material, apenas uma análise comparativa visual foi possível.

Nesse sentido, verificou-se que todos os gráficos apresentaram a mesma tendência de aumento ou redução do valor de interesse, na maioria os valores são aparentemente os mesmos para o modelo e o material de referência. Apesar disso, houve variações perceptíveis para os resultados de consumo específico de combustível e consumo de combustível, esses valores variam principalmente para rotações menores de 80% em que começam a ter problemas de pressão no bocal de saída dos gases quentes. Nesse ponto a pressão de entrada do bocal se torna menor que a atmosférica e foi optado por definir a velocidade de saída como 0 para poder calcular o restante dos valores.

---

### Exemplo 7.1: Turboélice - Calibrando o Modelo

#### Descrição do Exercício

Tomando os dados do motor PW127 ao nível do mar e estacionário: potência de eixo da turbina livre 2457 kW, vazão de ar no
compressor de 8,49 kg/s, razão de pressão do compressor 15,77 e temperatura de saída da câmara de combustão de 1600K. Fazer uma calibração do motor na simulação de desempenho para que se obtenha uma potência de eixo próxima à informada pelo fabricante.

Parâmetros de funcionamento do motor:

| Condição de funcionamento de referência     | Valores   |
| ------------------------------------------- | --------- |
| Vazão de massa                              | 8,49 kg/s |
| Temperatura de saída na câmara de combustão | 1600 K    |
| Razão de Pressão no Compressor              | 15.77     |


Condição de funcionamento:

| Condição de funcionamento         | Valores    |
| --------------------------------- | ---------- |
| Flight Mach                       | 0          |
| Pressão atmosférica               | 101,30 kPa |
| Temperatura atmosférica           | 288.15 K   |
| % de rotação máxima do compressor | 1,00       |


Eficiência e dados específicos sobre a relação de calor:

| Componente            | Eficiência Adiabática | $\gamma$ |
| --------------------- |:---------------------:| -------- |
| entrada               | 0,85                  | 1,40     |
| compressor            | 0,75                  | 1,37     |
| combustão             | 1,00                  | 1,35     |
| turbina do compressor | 0,90                  | 1,33     |
| Turbina livre         | 0,90                  | 1,33     |
| bocal de ar quente    | 0,98                  | 1,36     |

Dados termodinâmicos:

| Dados Termodinâmicos                    | Valores           |
| --------------------------------------- | ----------------- |
| PCI                                     | 45000 kJ/kg       |
| Média R                                 | 288,3 m^2/(s^2.K) |
| calor específico na câmara de combustão | 1,11 kJ/(kg.K)    |








#### Parte 1:  Ajuste da razão de pressão
Calibrar o modelo para o relatório do fabricante, variando a razão de pressão na turbina para atingir a potência da turbina especficiada de 2457 kW.

#### Resultados:
Variamos a taxa de expansão da turbina para encontrar a potência da turbina que corresponde à potência da turbina dada pelo fabricante. A imagem a seguir mostra os resultados obtidos:

<img src="images/questao1/ex7_1_a.jpg" style="width: 600px;"/>

A partir dos resultados acima, encontramos os seguintes parâmetros para calibrar o modelo:

| Parâmetros do Motor               | Valores |
| --------------------------------- | ------- |
| Razão de Pressão na Turbina Livre | 2.87    |
| % de rotação máxima no compressor | 1,00    |


#### Comparação de resultados

| Parâmetro                              | Unidade   | Modelo      | Referência do material | $\Delta$ em % |
| -------------------------------------- | --------- | ----------- | ---------------------- | ------------- |
| Razão de Pressão na Turbina Livre      | -         | 2.87        | 2.87                   | 0.000         |
| Press. Total na saída da Turbina livre | kPa       | 135.944     | 135.98                 | -0.021        |
| Temp. total na saída da Turbina livre  | K         | 931.58      | 931.58                 | 0.000         |
| Trabalho por vazão da Turbina livre    | kJ.s/kg   | 282.358     | 282,83                 | -0.161        |
| Vazão na turbina                       | kg/s      | 8.68121     | 8.69                   | -0.107        |
| Potência da turbina (termodinâmica)    | kW        | 2451.21     | 2456.49                | -0.210        |
| Velocidade de saída do bocal           | m/s       | 385.967     | 385.97                 | -0.002        |
| Empuxo Específico do bocal             | kN.s/kg   | 0.394757    | 0.395                  | -0.067        |
| Empuxo do bocal                        | kN        | 3.35066     | 3.352                  | -0.033        |
| Consumo Específico (BSFC)              | kg/(kN.s) | 7.88615e-05 | 7.87E-05               | 0.205         |
| Potência na caixa de redução           | kW        | 2402.19     | 2407.356               | -0.214        |
| Consumo Específico (EBSFC)             | kg/(kN.s) | 7.88615e-05 | 7.87E-05               | 0.205         |
| Consumo de combustível                 | kg/s      | 0.193306    | 0.193                  | 0.158         |




### Parte 2: Ajuste da Rotação

A potência termodinâmica da turbina é superior à potência limite da caixa de redução de 2050 kW.
Assim, a solução consiste em reduzir a rotação do compressor.

A imagem a seguir mostra a variação da potência em função da rotação do motor, para determinar a rotação máxima correspondente ao limite da caixa de redução:

<img src="images/questao1/ex7_1_b.jpg" style="width: 600px;"/>

Parâmetros do motor variados para atingir o desempenho do fabricante:


| Parâmetros do motor               | Valores |
| --------------------------------- | ------- |
| Razão de Pressão na Turbina Livre | 2.87    |
| % de rotação máxima no compressor | 0,941   |

#### Comparação de resultados

| Parâmetro                              | Unidade   | Modelo      | Referência do material | $\Delta$ em % |
| -------------------------------------- | --------- | ----------- | ---------------------- | ------------- |
| % da rotação máxima do compressor      | -         | 0.941       | 0.941                  | 0.0000        |
| Razão de Pressão na Turbina Livre      | -         | 2.87        | 2.87                   | 0.0000        |
| Temp. total na saída da Camara         | K         | 1491.42     | 1491.87                | -0.0301       |
| Press. Total na saída da Camara        | kPa       | 1446.44     | 1447.03                | -0.0407       |
| Razão combustivel/ar                   | -         | 0.0206016   | 0.02061                | -0.0407       |
| Press. Total na saída da Turbina livre | kPa       | 119.654     | 119.68                 | -0.0217       |
| Temp. total na saída da Turbina livre  | K         | 863.739     | 863.96                 | -0.0255       |
| Trabalho por vazão da Turbina livre    | kJ.s/kg   | 264.167     | 264.69                 | -0.1975       |
| Vazão na turbina                       | kg/s      | 7.74105     | 7.75                   | -0.1154       |
| Potência da turbina (termodinâmica)    | kW        | 2044.93     | 2050.13                | -0.2536       |
| Velocidade de saída do bocal           | m/s       | 281.973     | 282.18                 | -0.0733       |
| Empuxo Específico do bocal             | kN.s/kg   | 0.287782    | 0.288                  | -0.0756       |
| Empuxo do bocal                        | kN        | 2.18277     | 2.186                  | -0.1477       |
| Consumo Específico (BSFC)              | kg/(kN.s) | 7.64128e-05 | 7.63e-05               | 0.1478        |
| Potência na caixa de redução           | kW        | 2004.03     | 2009.131               | -0.2538       |
| Consumo de combustível                 | kg/s      | 0.156259    | 0.156                  | 0.1660        |

---

### Exemplo 7.2: Turbohélice - Simulação de Condição de Vôo

#### Descrição do exercício

Simular agora o motor operando a 7.000 m e Mach 0,45, mantendo outros parâmetros de entrada constante em relação ao Exemplo 7.1. Como estratégia de operação considerar N1 =  100% e N2 = 85%.

Parâmetros de funcionamento do motor:

| Condição de funcionamento da linha de base  | Valores   |
| ------------------------------------------- | --------- |
| Fluxo de massa                              | 8,49 kg/s |
| Temperatura de saída na câmara de combustão | 1600 K    |
| Razão de Pressão no Compressor              | 15.77     |


Condição de funcionamento:

| Condição de operação de voo       | Valores   |
| --------------------------------- | --------- |
| Mach                              | 0.45      |
| Pressão atmosférica               | 41,00 kPa |
| Temperatura atmosférica           | 246.55 K  |
| % de rotação máxima do compressor | 0,85      |


Eficiência e dados específicos sobre a relação de calor:

| Componente            | Eficiência Adiabática | $\gamma$                  |
| --------------------- |:---------------------:| ------------------------- |
| entrada               | 0,85                  | 1,40                      |
| compressor            | 0,75                  | 1,37                      |
| combustão             | 1,00                  | 1,35                      |
| turbina do compressor | 0,90                  | 1,33                      |
| turbina livre         | 0,90                  | 1,33                      |
| bocal de ar quente    | 0,98                  | 1,36                      |

Dados termodinâmicos:

| Dados Termodinâmicos                    | Valores           |
| --------------------------------------- | ----------------- |
| PCI                                     | 45000 kJ/kg       |
| Média R                                 | 288,3 m^2/(s^2.K) |
| Cp na câmara de combustão | 1,11 kJ/(kg.K)    |







#### Comparação de resultados

| Parâmetro                              | Unidade   | Modelo      | Referência do material | $\Delta$ em % |
| -------------------------------------- | --------- | ----------- | ---------------------- | ------------- |
| Press. Total na saída da Turbina livre | kPa       | 55.8689     | 55.87                  | -0.0019       |
| Temp. total na saída da Turbina livre  | K         | 798.767     | 798.77                 | -0.0003       |
| Trabalho por vazão da Turbina livre    | kJ.s/kg   | 221.816     | 222.19                 | -0.1683       |
| Vazão na turbina                       | kg/s      | 2.85347     | 2.85                   | 0.1217        |
| Potência da turbina (termodinâmica)    | kW        | 632.945     | 634.00                 | -0.1664       |
| Velocidade de saída do bocal           | m/s       | 366.196     | 366.20                 | -0.0010       |
| Empuxo Específico do bocal             | kN.s/kg   | 0.231       | 0.231                  | 0.0000        |
| Empuxo do bocal                        | kN        | 0.647313    | 0.647                  | 0.0483        |
| Consumo Específico no eixo (BSFC)      | kg/(kW.s) | 8.29033e-05 | 8,28E-05               | 0.1247        |
| Potência na caixa de redução           | kW        | 620.286     | 621.323                | -0.1669       |
| Empuxo produzido pela hélice           | kN        | 3.71415     | 3.720                  | -0.1572       |
| Empuxo total                           | kN        | 4.36146     | 4.368                  | -0.1497       |
| Consumo Específico equivalente(EBSFC)  | kg/(kW.s) | 7.23934e-05 | 7.23e-05               | 0.1291        |
| Consumo Específico empuxo (TSFC)       | kg/(kN.s) | 0.0120311   | 0.012                  | 0.2591        |
| Consumo de combustível                 | kg/s      | 0.0524732   | 0.052                  | 0.9100        |



---

### Exemplo 7.3: TurboFan - Comparando ao Turboélice (Ex. 7.2)

#### Descrição do exercício

Simular um motor turbofan para cumprir a mesma missão do motor turbo hélice do exemplo 7.2, ou seja, gerar o empuxo de 4.26 kN a 7.000 m e a Mach de voo 0,45. Para fazer as simulação tomou-se como referência dados do motor PW545B, com razão de pressão total de 12,5 e razão de passagem 4,2 na condição de decolagem ao nível do mar. Algumas hipóteses foram feitas para simulação como vai ser discutido na sequência.

Parâmetros de funcionamento do motor:

| Parametros de funcionamento do motor        | Valores |
| ------------------------------------------- | ------- |
| Temperatura de saída na câmara de combustão | 1550 K  |
| Razão de Pressão no Compressor              | 8.33    |
| Razão de Pressão em Booster                 | 1.00    |
| Razão de Pressão no fan                     | 1.50    |
| Razao de passagem de ar                     | 4.2     |
| Perda de pressão na câmara de combustão     | 0 %     |

Condição de funcionamento:

| Condição de operação de voo       | Valores   |
| --------------------------------- | --------- |
| mach                              | 0.45      |
| Pressão atmosférica               | 41,00 kPa |
| Temperatura atmosférica           | 246.55 K  |
| % de rotação máxima do compressor | 0,85      |


Eficiência e dados específicos sobre a relação de calor:

| Componente            | Eficiência Adiabática | $\gamma$ |
| --------------------- |:---------------------:| ------------------------- |
| entrada               | 0,97                  | 1,40                      |
| fan                   | 0,90                  | 1,40                      |
| compressor            | 0,75                  | 1,37                      | 
| combustão             | 0,9995                | 1,35                      |
| turbina do compressor | 0,90                  | 1,33                      |
| Turbina do fan        | 0,90                  | 1,33                      |
| Bocal de ar quente    | 0,98                  | 1,36                      |
| Bocal do fan          | 0,98                  | 1,40                      |

Dados termodinâmicos:

| Dados Termodinâmicos                    | Valores           |
| --------------------------------------- | ----------------- |
| PCI                                     | 45000 kJ/kg       |
| Média R                                 | 288,3 m^2/(s^2.K) |
| Cp na câmara de combustão | 1,11 kJ/(kg.K)    |

Os parâmetros do motor variaram para alcançar o exemplo do TurboProp 7.2 de desempenho:

| Parâmetros do motor | Valores    |
| ------------------- | ---------- |
| Fluxo de massa      | 88,42 kg/s |


#### Comparação de resultados

| Parâmetro                        | Unidade   | Modelo    | Referência do material | $\Delta$ em % |
| -------------------------------- | --------- | --------- | ---------------------- | ------------- |
| Empuxo Específico do bocal       | kN.s/kg   | 0.926056  | 0.8849                 | 4.65092       |
| Vazão massica                    | kg/s      | 29.1712   | 29.18                  | -0.03015      |
| Vazão massica do gerador de gas  | kg/s      | 24.3576   | 24.365                 | -0.03037      |
| Vazão massica do bocal de fan    | kg/s      | 4.81367   | 4.814                  | -0.00685      |
| Empuxo total                     | kN        | 4.45773   | 4.26                   | 4.64154       |
| Consumo Específico empuxo (TSFC) | kg/(kN.s) | 0.020879  | 0.02186                | -4.48764      |
| Consumo de combustível           | kg/s      | 0.0930729 | 0.09313                | -0.06131      |

---

## Exercício 3: Evolução dos Motores Turbofan

* Eloi Antonio
* Gabriel Henrique


Deseja-se produzir 100 kN no início da decolagem (Mach = 0) ao nível do mar (Ta =
288,2K e Pa = 101,3 kPa) usando motores turbofans com tecnologias de diferentes
décadas, conforme tabela abaixo (ponto de projeto estático ao nível do mar).

### A) Compare:
 Compare a vazão de ar, consumo de combustível, consumo específico de combustível e empuxo específico (parâmetros de desempenho) na condição de projeto, estática ao nível do mar.

#### Geração dos resultados 

Para a resolução deste exercício são utilizados os valores de eficiência dos componentes, constante do gás para o ar e poder calorífico do combustível definidos para o Exercício 2. Para garantir que os diferentes motores gerassem o empuxo total igual a 100 kN na condição estática a nível do mar foram calibrados os valores de vazão mássica de ar na entrada para cada motor. Os resultados são indicados nas tabelas e gráficos seguintes:

***Calibração da vazão mássica de ar***

| Década do motor   |    Vazão mássica de ar na entrada [kg/s] |    Empuxo total [kN] |
|:-----------------:|:----------------------------------------:|:--------------------:|
| 60/70             |                                  266.338 |                  100 |
| 70/80             |                                  306.021 |                  100 |
| 80/90             |                                  318.217 |                  100 |
| 90/00             |                                  343.206 |                  100 |
| 00/10             |                                  356.071 |                  100 |

***Comparação dos parâmetros de desempenho entre os motores***

|  Década do motor     |   Vazão mássica de gases quentes [kg/s] |   Consumo de combustível [kg/s] |   TSFC [kg/(kN s)] |   Empuxo Específico [(kN s)/kg] |
|:-----:|:---------------:|:------------------:|:------:|:-----------------:|
| 60/70 |          88.779 |              1.491 |  0.015 |             1.126 |
| 70/80 |          61.204 |              1.179 |  0.012 |             1.634 |
| 80/90 |          53.036 |              1.083 |  0.011 |             1.886 |
| 90/00 |          38.134 |              0.951 |  0.01  |             2.622 |
| 00/10 |          32.37  |              0.874 |  0.009 |             3.089 |

***Gráficos contendo os parâmetros de desempenho***

<img src="images/questao3/itemA.jpg" alt="Fluxo mássico e consumo de combustível das diferentes geracoes de motores variando N2" style="width: 700px;"/>

#### Análise dos resultados

A partir dos dados é possível concluir que houve um aumento da eficiência na geração de empuxo ao modernizar os motores turbofan. Isso pode ser confirmado quando verificado que para um mesmo valor de empuxo (nesse exercício o empuxo total gerado foi de 100kN) menores valores de consumo de combustível específico e maiores valores do empuxo específico são obtidos a medida em que são analisados motores de décadas subsequentes.
 
Ademais é possível constatar a diminuição do fluxo mássico de gases quentes a medida que se avança em gerações subsequentes de motores turbofan. Que pode ser explicado pelos seguintes fatores:
- Aumento da razão de passagem é possibilitado um aumento na geração do empuxo sem a necessidade de aumento da vazão mássica que adentra o interior do motor; 
- Aumento da razão de pressão do compressor possibilitando que um menor fluxo mássico de ar de gases quentes seja capaz de produzir uma quantidade maior de empuxo.

### B) Variação da rotação do compressor na condição estática a nível do mar
Ainda na condição estática e ao nível do mar, varie a rotação até 70% de N2 e
compare como cada motor se comporta sobre os parâmetros de desempenho e
empuxo em valores absolutos e normalizados pela condição de 100% de N2.

#### Geração dos resultados 

Partindo dos valores de projeto definidos no item "a", foram variadas os valores de velocidade de rotação do compressor (N2) e os índices de desempenho de cada motor nessas condições foram avaliadas. A seguir são apresentados os gráficos contendo os resultados encontrados, tanto em valores absolutos quanto em valores normalizados. Essa normalização foi realizada tendo como referência os valores no ponto de projeto (N2 = 100%) calculados no item "a".

***Gráficos contendo os parâmetros de desempenho para motores***


<img src="images/questao3/itemB_1.jpg" alt="Fluxo mássico e consumo de combustível das diferentes geracoes de motores variando N2" style="width: 700px;"/>

<img src="images/questao3/itemB_2.jpg" alt="TSFC e Empuxo Específico das diferentes geracoes de motores variando N2" style="width: 700px;"/>

#### Análise dos resultados

A partir dos gráficos é possível verificar que conforme a rotação do compressor é diminuída, há uma menor vazão de ar quente no motor, visto que há uma relação diretamente proporcional entre a rotação do compressor e a quantidade de fluxo de ar que admite. Ademais, também é observada uma redução do consumo de combustível com a redução da rotação, a qual inclusive é mais acentuada do que a queda de vazão mássica de ar quente. Esta redução do consumo de combustível ocorre, primeiramente, por conta da redução da vazão mássica de ar quente no motor, que implica na diminuição da necessidade de injeção de combustível para ignitar o ar. Mas a redução do consumo de combustível também ocorre por questões de manutenção do motor: como o motor não está operando em máxima potência quando a rotação do compressor é reduzida, a câmara de combustão do motor não precisa operar na mesma temperatura em que opera na condição de rotação máxima, o que é desejável para que reduzir a danificação dos componentes da parte quente do motor; e para ter essa diminuição de temperatura, a razão ar/combustível precisa ser reduzida, o que implica na redução mais acelerada do consumo de combustível.

No que diz respeito à comparação entre diferentes gerações de motores, é possível perceber que os motores desenvolvidos nas décadas mais recentes apresentam uma melhora significativa na quantidade de fluxo de ar e de combustível consumido. Porém, o comportamento normalizado em função da diminuição de rotação é similar para todas as gerações, com uma certa vantagem para as gerações mais novas.

A partir do gráfico de consumo específico de combustível (TSFC) é observado um comportamento não-linear à medida que a variação de rotação do compressor. Nisto com a diminuição da rotação até cerca de 85% da rotação de N2 em ponto de projeto o parâmetro TSFC é levemente reduzido, mas ao diminuir ainda mais N2, ocorre o comportamento inverso com o aumento de TSFC. Isso pode ser explicado sabendo que a nas rotações próximas ao ponto de projeto operação do motor é mantida com uma eficiência praticamente constante, mas nos regimes de menor rotação, a eficiência do motor começa a ser comprometida, resultando na necessidade de maior TSFC.

E com o aumento do TSFC aliado à redução da razão combustível-ar, o empuxo específico (que consiste na divisão entre a razão combustível-ar e o TSFC) acaba sofrendo uma redução com a diminuição da rotação do compressor. Inicialmente, a rotações próximas do ponto de projeto, o empuxo específico, sofre menores reduções, visto que o TSFC praticamente não sofre alteração e, assim, não interfere na perda de empuxo específico. Porém, à medida que o TSFC vai sendo aumentado, uma vez que diminui-se a rotação N2, o empuxo específico passa a ter uma taxa de redução ainda mais acentuada.

Porém, para os modelos de motores turbofan que apresentam alta razão de passagem (B>5) são percebidos pontos de inflexão tanto no gráfico de empuxo específico x rotação de N2 quanto de TSFC x rotação de N2. Esses pontos revelam uma limitação no modelo para representar o comportamento de motores com alta razão de passagem. Ao qual está relacionada a diminuição da pressão no bocal de saída dos gases quentes chegando esta a ser menor que a pressão atmosférica o que resulta em valores de velocidade de saída do bocal de gases quentes não condizentes (valores NaN) com o observado fisicamente, assim se pode concluir para o motor da década de 2000-2010 os valores disponíveis nos gráficos são validos de 100% até 85% de N2 e para o motor da década de 1990/2000 existe validade dos dados de 100% até aproximadamente 80% de N2. 

Portanto comparando os motores de diferentes gerações, é possível notar que os motores desenvolvidos nas décadas mais recentes, são mais eficientes tanto no consumo de combustível (já que apresentam menor TSFC) e na geração de empuxo específico. Finalmente no que se refere ao TSFC, também é observado que os motores turbofan mais novos são capazes de se manter mais eficientes a velocidades de rotação do compressor mais baixas, o que é comprovado pelo menor aumento do TSFC.

### C) Parâmetros de desempenho a voo de cruzeiro
Pensando no voo de cruzeiro a Mach 0,85 e 12.200m ((Ta = 216,7K e Pa =
18,75 kPa) e que os motores operam a 85% de N2, calcule os parâmetros de
desempenho anteriores, somando-se a eles o empuxo gerado.

#### Geração dos resultados 

Nesse exercício manteve-se os valores de eficiências dos componentes do motor, de $\gamma$, da constantes do gás e poder calorífico do combustível  do item "a".

Porém para o cálculo dos parâmetros de desempenho nas condições estabelecidas foi realizada uma nova avaliação de operação dos motores a voo de cruzeiro (M=0.85, Ta=216,7 K e Pa= 18,75 kPa) e com uma rotação do compressor igual a 85% do valor de projeto.

Ademais foram atualizadas as vazões mássicas de ar para a altitude de 12.200 m. Dessa maneira são disponibilizados na tabela e gráficos seguintes os resultados obtidos:

**Comparação entre os motores turbofan de diferentes décadas**
|  Década do motor     |   Vazão mássica de gases quentes [kg/s] |   Consumo de combustível [kg/s] |   TSFC [Kg/(kN s)] |   Empuxo Específico [(kN s)/kg] | Empuxo total [kN] | N2 |
|:-----:|:---------------:|:------------------:|:------:|:-----------------:|:--------------:|:----:|
| 60/70 |          13.415 |              0.201 |  0.026 |             0.583 |          7.822 | 0.85 |
| 70/80 |           9.031 |              0.154 |  0.022 |             0.76  |          6.867 | 0.85 |
| 80/90 |           7.78  |              0.141 |  0.021 |             0.849 |          6.609 | 0.85 |
| 90/00 |           5.54  |              0.121 |  0.02  |             1.09  |          6.038 | 0.85 |
| 00/10 |           4.687 |              0.111 |  0.019 |             1.237 |          5.797 | 0.85 |

***Gráficos contendo os parâmetros de desempenho***

<img src="images/questao3/itemC_1.jpg" alt="Parametros de desempenho das diferentes geracoes de motores" style="width: 700px;"/>

<img src="images/questao3/itemC_2.jpg" alt="Empuxo gerado pelas diferentes geracoes de motores" style="width: 700px;"/>

#### Análise dos resultados

Com o aumento da altitude de voo é possível identificar um aumento do TSFC e diminuição do empuxo específico, o que é consequência da queda de massa específica do ar e do fluxo de quantidade de movimento na entrada do motor (que é acrescido com a razão de passagem e velocidade de voo). Este último é diminuido do empuxo gerado pela exaustão dos gases nas saídas dos bocais.

A redução verificada na vazão mássica de gases quentes é consequência direta da diminuição da vazão da massa de ar que entra no motor que por sua vez é resultado da diminuição da massa específica do ar atmosférico em maiores altitudes. 

Similarmente ao verificado no item "a" ocorre a diminuição do consumo de combustível, uma vez que há menor necessidade de sua injeção na mistura ar-combustível.

Em relação ao empuxo gerado nas condições de vôo em cruzeiro é possível notar que há uma redução do seu valor quando comparado com o valor verificado em condições á nível do mar. Além disso os motores mais recentes necessitam gerar menos empuxo para manter a aeronave nas condições de voo de cruzeiro o que corrobora com a maior eficiência de gerações mais novas de motores.


In [2]:
!jupyter nbconvert --to html --no-input --TemplateExporter.exclude_output_prompt = True Final_Report.ipynb


[NbConvertApp] WARNING | pattern 'True' matched no files
[NbConvertApp] Converting notebook Final_Report.ipynb to html
[NbConvertApp] Writing 645110 bytes to Final_Report.html
